In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer

from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding

In [41]:
data = pd.read_csv('dataset/mail_data.csv')
data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [42]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['Spam'] = le.fit_transform(data['Category'])
data = data.drop('Category',axis=1)
data.head()

,Message,Spam
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


In [44]:
X = data['Message']
y = data['Spam']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y)

In [45]:
from nltk.tokenize import word_tokenize
import re
import string
def remove_hyperlink(word):
    return  re.sub(r"http\S+", "", word)
def to_lower(word):
    result = word.lower()
    return result
def remove_number(word):
    result = re.sub(r'\d+', '', word)
    return result
def remove_punctuation(word):
    result = word.translate(str.maketrans(dict.fromkeys(string.punctuation)))
    return result
def remove_whitespace(word):
    result = word.strip()
    return result
def replace_newline(word):
    return word.replace('\n','')

In [46]:
def clean_up_pipeline(sentence):
    cleaning_utils = [remove_hyperlink,
                      replace_newline,
                      to_lower,
                      remove_number,
                      remove_punctuation,remove_whitespace]
    for o in cleaning_utils:
        sentence = o(sentence)
    return sentence

In [47]:
X_train = [clean_up_pipeline(o) for o in X_train]
X_test = [clean_up_pipeline(o) for o in X_test]

In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score,precision_score
tfidf = TfidfVectorizer()
feautre_extraction = TfidfVectorizer(min_df=1,stop_words='english',lowercase='True')
X_train_features = feautre_extraction.fit_transform(X_train)
X_test_features = feautre_extraction.transform(X_test)

y_train = y_train.astype('int')
y_test = y_test.astype('int')

In [61]:
from sklearn.naive_bayes import MultinomialNB
mb = MultinomialNB()
mb.fit(X_train_features,y_train)

MultinomialNB()

In [63]:
pred = mb.predict(X_test_features)
accuracy = accuracy_score(y_test,pred)
precision = precision_score(y_test,pred)

In [64]:
accuracy,precision

(0.9547738693467337, 1.0)

In [65]:
input_mail = ["WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only."]
input_data_features = feautre_extraction.transform(input_mail)
prediction = mb.predict(input_data_features)
print(prediction)

[1]


In [66]:
import pickle

pickle.dump(mb,open('model.pkl','wb'))
pickle.dump(feautre_extraction,open('vectorizer.pkl','wb'))